
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="files/image/DD_07.png" alt="Databricks Learning" style="width: 1500px">
</div>


## MLflow


### 학습 목표 
- MLflow의 Logging API 사용하여 log 관리 기능을 알아봅니다.

In [0]:
# Libraries import
import mlflow
import mlflow.sklearn
import pandas as pd
import matplotlib.pyplot as plt

from numpy import savetxt

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error




In [0]:
# 두 개의 카테고리로 데이터 분할 ,Training data , Testing data

db = load_diabetes()
x = db.data
y = db.target

X_train, X_test, y_train, y_test = train_test_split(x, y)

In [0]:
print(X_train)

In [0]:
print(X_test)

In [0]:
print(y_train)

In [0]:
print(y_test)

In [0]:
print("Number of observations in the training data:" ,len(X_train))
print("Number of observations in the test data:" ,len(X_test))
print("Number of observations in the test data:", len(y_train))
print("Number of observations in the test data:", len(y_test))

#### MLflow version 확인

In [0]:
# mlflow version 확인
# version 2.1.1, 12.2 LTS 이상 부터  pre- built in 
import mlflow
print(mlflow.version.VERSION)

랜덤 포레스트 회귀(Random Forest Regressor)는 여러 개의 결정 트리(Decision Tree)를 결합하여 예측 성능을 향상시키는 앙상블 학습 방법입니다. \
각 트리는 데이터의 무작위 하위 샘플을 사용하여 학습되고, 최종 예측은 모든 트리의 예측을 평균하여 결정됩니다

하이퍼파라미터(Hyperparameter)는 머신러닝 모델의 학습 과정에서 사용자가 직접 설정해야 하는 매개변수입니다. \
모델이 데이터를 통해 학습하는 파라미터(Parameter)와는 달리, 하이퍼파라미터는 모델의 구조나 학습 방법을 정의하는 데 사용됩니다

In [0]:
# model Training build model
# Feature: max_depth
# random forest regressor 알고리즘 사용

# hyper parameter 설정
n_estimators = 100
max_depth = 6
max_features = 3

#모델 생성
rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
rf.fit(X_train, y_train)

# Test dataset을 사용하여 예측
predictions = rf.predict(X_test)

### Experiment에서 생성된 Model 을 확인 할 수 있습니다.

In [0]:
#auto log 활성화

#tensorflow,sklern Pytorch 등의 Library를 사용하여 자동으로 log를 생성 할 수 있습니다.
mlflow.sklearn.autolog()

# autolog는 모든 model parameter, model score, fitted model 을 자동으로 기록합니다.
with mlflow.start_run() as run:


    # parameter 설정
    n_estimators = 100
    max_depth = 6
    max_features = 3


    # train model 생성
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)

    # Testdataset 에서 모델을 사용하여 예측 생성.
    predictions = rf.predict(X_test)



모델을 하나 만들 때 마다 새로운 Experiment 가 생성되는 것이 아니라 같은 노트북에서는  같은 experiment에서 새로운 모델을 만드는 것 입니다.\
따라서 특정 노트북내에서 모델을 생성하는 코드를 실행하면 하나의 Experiment 에서 로그가 기록됩니다.

위의 셀을 parameter만 바꾸어서 실행하면 같은 Experiment에서 새로운 모델이 생기는 것 확인 할 수 있습니다.

In [0]:
#auto log 활성화

#tensorflow,sklern Pytorch 등의 Library를 사용하여 자동으로 log를 생성 할 수 있음.
mlflow.sklearn.autolog()

# autolog는 모든 model parameter, model score, fitted model 을 자동으로 기록한다.

with mlflow.start_run() as run:


    # parameter 설정
    # n_estimators = 100
    n_estimators = 120
    max_depth = 6
    max_features = 3


    # train model 생성
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)

    # Testdataset 에서 모델을 사용하여 예측 생성.
    predictions = rf.predict(X_test)

n_estimators 와 같이 달라진 parameter를 포함해서 모든 것 들이 기록됩니다.

### Model Artifacts

Model artifact Model Artifact는 머신러닝 모델과 관련된 모든 파일과 데이터를 포함하는 패키지입니다. 이 패키지는 모델을 재현하거나 배포할 때 필요한 모든 정보를 담고 있습니다. \
 주요 구성 요소는 다음과 같습니다:  

* 모델 파일: 실제 머신러닝 모델이 저장된 파일입니다. 예를 들어, PyTorch 모델의 경우 .pt 파일이 될 수 있습니다.
* 환경 설정 파일: 모델을 실행하는 데 필요한 라이브러리와 패키지를 정의한 requirements.txt 또는 conda.yaml 파일입니다.
* 추가 아티팩트: 모델의 성능을 평가하는 데 사용된 메트릭, 플롯, 기타 관련 파일들이 포함될 수 있습니다. 

이를 통해 모델 개발 환경을 재현할 수 있습니다1.


 Estimator는 머신러닝 모델을 학습시키기 위한 객체입니다. Estimator는 데이터셋을 입력으로 받아 모델을 학습시키고, 학습된 모델을 반환합니다

### Experiment에 Artifact 파일들

- 모델은 model.py 에 있습니다.
- conda.yaml: Conda의 환경을 정의하고 공유하는 데 사용되는 파일.특정 환경에서 필요한 패키지와 그 버젼 정보가 포함되어 있습니다. 
  * Conda는 패키지 및 환경을 관리하는 데이터 과학 및 ML 프로젝트에서 사용되는 패키지 및 환경관리도구 입니다.
- python_env: dependency를 포함하고 있는 python 설정파일입니다.
- requirements.txt: library dependency를 정보를 포함하고 있습니다.
- estomator.html: 다운로드 가능한 estimator dot HTML file 입니다.


## Default experiment 외 다른 Experiment 실행하기

In [0]:
experiment_name = "/Users/hanki.kim@data-dynamics.io/diabetes_experiment"
mlflow.set_experiment(experiment_name)

with mlflow.start_run() as run:


    # parameter 설정
    # n_estimators = 100
    n_estimators = 120
    max_depth = 6
    max_features = 3

    #Log parameter 생성
    mlflow.log_param("num_trees", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)


    # train model 생성
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)

    # Testdataset 에서 모델을 사용하여 예측 생성.
    predictions = rf.predict(X_test)

#### 새로운 experiment가 생성 되는 것을 확인 할 수 있습니다.

## Experimnet의 이름을 알고 있을 때, 특정  Experiment 의 정보 가져오기

In [0]:
import mlflow

experiment = mlflow.set_experiment("/Users/hanki.kim@data-dynamics.io/diabetes_experiment")


print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags : {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))

Experiments의 메뉴에서 해당 Experiment의 Overview 에서도 위의 정보들을 확인 할 수 있습니다.

# experiment_id 로  Experiment 실행하기

`mlflow.start_run(experiment_id="")` 부분에 **experiment_id**  입력 후 실행

In [0]:
with mlflow.start_run(experiment_id = "") as run:


    # parameter 설정
    # n_estimators = 100
    n_estimators = 120
    max_depth = 6
    max_features = 3

    #Log parameter 생성
    mlflow.log_param("num_trees", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("max_features", max_features)


    # train model 생성
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features)
    rf.fit(X_train, y_train)

    # Testdataset 에서 모델을 사용하여 예측 생성.
    predictions = rf.predict(X_test)